In [8]:
from run_capsule import *
from decoding_utils import *
import sys
sys.argv[1:] = ['--result_prefix', 'debug']
params = Params()

In [14]:
units = get_filtered_units(params).collect()

In [18]:
units = utils.get_df('units', lazy=False)

In [21]:
params.units_group_by

[<Expr ['col("session_id")'] at 0x7EFC096D8B80>,
 <Expr ['col("structure")'] at 0x7EFC096D9F90>,
 <Expr ['col("electrode_group_name")'] at 0x7EFC096D9720>]

In [31]:
params.units_query

<Expr ['[([([(col("isi_violations_rati…'] at 0x7EFC096DA080>

In [33]:
(
    units
    .pipe(group_structures, keep_originals=True)
    .filter(
        params.units_query,
        pl.col('isi_violations_ratio').le(0.5),
        pl.col('presence_ratio').ge(0.9),
        pl.col('amplitude_cutoff').le
    )
    .filter(
        pl.col('unit_id').n_unique().ge(params.n_units).over(params.units_group_by),
    )
    .group_by(*params.units_group_by)
    .agg(
        pl.col('unit_id').n_unique().alias('n_units'),
    )
    .sort('n_units')
)


session_id,structure,electrode_group_name,n_units
str,str,str,u32
"""703333_2024-04-10""","""out of brain""","""probeB""",1
"""644864_2023-01-31""","""out of brain""","""probeA""",1
"""699847_2024-04-16""","""out of brain""","""probeB""",1
"""636397_2022-09-27""","""out of brain""","""probeF""",1
"""626791_2022-08-16""","""out of brain""","""probeC""",1
…,…,…,…
"""746439_2025-01-29""",null,"""probeE""",537
"""670180_2023-07-25""","""ProS""","""probeC""",552
"""744740_2024-11-13""",null,"""probeE""",584


In [17]:
units.group_by(*params.units_group_by).agg(pl.col('unit_id').n_unique().alias('n_units')).sort('n_units')

session_id,structure,electrode_group_name,n_units
str,str,str,u32
"""626791_2022-08-16""","""out of brain""","""probeC""",1
"""742903_2024-10-23""","""DG""","""probeE""",1
"""733780_2024-08-26""","""TTd""","""probeF""",1
"""699847_2024-04-16""","""out of brain""","""probeB""",1
"""636766_2023-01-25""","""out of brain""","""probeE""",1
…,…,…,…
"""746439_2025-01-29""",null,"""probeE""",537
"""670180_2023-07-25""","""ProS""","""probeC""",552
"""744740_2024-11-13""",null,"""probeE""",584
